In [1]:
import tensorflow as tf

In [3]:
class LogisticClassifier:
    def __init__(self, seq_max_len,state_size, vocab_size, num_classes):
        #init de los parametros con el constructor de la clase
        self.seq_max_len = seq_max_len
        self.state_size = state_size
        self.vocab_size = vocab_size
        self.num_classes = num_classes
        
    def build_model(self):
        #el placeholder se utiliza para definir el proceso de entrada de las features
        self.x = tf.placeholder(shape = [None, self.seq_max_len], dtype= tf.int32)
        #se utiliza 'one hot encoding' para convertir valores categóricos a numéricos
        x_one_hot = tf.one_hot(self.x, self.vocab_size)
        x_one_hot = tf.cast(x_one_hot, tf.float32)
        self.y = tf.placeholder(shape=[None], dtype= tf.int32)
        self.y_one_hot = tf.one_hot(self.y, self.num_classes, dtype= tf.float32)
        self.batch_size= tf.placeholder(tf.int32, [], name= 'batch_size')
        #al ser un modelo sin capas ocultas, se considera lineal, por ende sigue la formula
        # y = mx + b donde y es el resultado, m es la matriz de pesos y x es el vector de inputs
        weights = {
            
            'layer_0': tf.Variable(tf.random_normal([self.seq_max_len*self.vocab_size, self.num_classes]))
        }
        biases = {
            
            'layer_0': tf.Variable(tf.random_normal([self.num_classes]))
        }
        
        x_input =tf.reshape(x_one_hot, [-1, self.seq_max_len*self.vocab_size])
        #se realiza la operación feedforward y=mx + b
        output = tf.matmul(x_input, weights['layer_0']) + biases['layer_0']
        self.logits = tf.sigmoid(output)
        self.probs = tf.nn.softmax(self.logits, axis=1)
        self.correct_preds = tf.equal(tf.argmax(self.probs,axis= 1),tf.argmax(self.y_one_hot, axis = 1))
        self.acc= tf.reduce_mean(tf.cast(self.correct_preds), tf.float32())  
        
        
    def entrenamiento(self, learning_rate =0.01):
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits= self.logits, labels= self.y_one_hot))
        
        optimizador = tf.train.AdamOptimizer(learning_rate= learning_rate).minimize(loss)
        
        return  loss, optimizador
        
        